In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark").getOrCreate()


In [ ]:
spark.version

In [ ]:
spark.range(5).show()

In [ ]:
strings = spark.read.text("lorem.md")
strings.show(3, truncate=False)

In [ ]:
strings.count()

In [ ]:
filtered = strings.filter(strings.value.contains("eget"))
filtered.count()

In [ ]:
dfhr = spark.read.csv("emp.csv", header=True, inferSchema=True)
dfhr.show()

In [ ]:
dfhr.printSchema()

In [ ]:
dfhr.select("name").show()

In [ ]:
dfhr.filter(dfhr["salary"] > 5000).show()

In [ ]:
dfhr.filter(dfhr.salary > 5000).show()

In [ ]:
dfhr.filter(dfhr.dep_id == 1).show()

In [ ]:
dfhr.filter(dfhr.dep_id == 1).select("name","salary").orderBy("salary", ascending=False).show()

In [ ]:
dfdep = spark.read.csv("dep.csv", header=True, inferSchema=True)
dfdep.show()

In [ ]:
dfhr.join(dfdep, dfhr.dep_id == dfdep.id,"inner").show()

In [ ]:
dfhr.printSchema()
dfdep.printSchema()

In [ ]:
from pyspark.sql.functions import avg

dfhr.groupBy("dep_id").agg(avg("salary")).show()

In [ ]:
from pyspark.sql.functions import concat_ws

dfhr.select(concat_ws(" ", dfhr.name, dfhr.salary).alias("label")).show()

In [ ]:
pdf = dfhr.select("salary","dep_id").toPandas()

import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x="dep_id", y="salary", data=pdf)
plt.show()

In [ ]:
dfhr.explain()

In [ ]:
dfhr.limit(2).join(dfdep.limit(2), dfhr.dep_id == dfdep.id, "inner").show()

In [ ]:
dfhr.filter(dfhr.dep_id.isNotNull()).show()

In [ ]:
dfhr.printSchema()

In [ ]:
from pyspark.sql.functions import col

dfhrcast = dfhr.withColumn("dep_id", col("dep_id").cast("string"))
dfhrcast.printSchema()

In [ ]:
dfhrcast.show(5)

In [ ]:
from pyspark.sql.functions import when

dfhr.select("name", when(dfhr.salary > 5000, "high").otherwise("low").alias("salary_group")).show()

In [ ]:
dfhr.groupBy("dep_id") \
.agg(count("*").alias("count")) \
.toPandas() \
.to_csv("dep_count.csv", index=False)

In [ ]:
from pyspark.sql.functions import count, first

df = dfhr.join(dfdep, dfhr.dep_id == dfdep.id,"inner") \
.groupBy(dfhr.dep_id) \
.agg(
    count("*").alias("count"),
    first(dfdep.name).alias("name")
) \
.toPandas()
df["label"] = df["dep_id"].astype(str) + " - " + df["name"]

df.to_csv("dep_count.csv", index=False)
